In [1]:
import crud_operations as co
import mongo_operations as m
import sqlparse
from itertools import islice
import re

In [2]:
def processQueries():
    db = co.MySQLDB('pl_queries')
    
    queries = {}
    res = db.readAllQueries()
    for q in res:
        queries[(q[0], q[1], q[2])] = q[3]
    return queries

In [3]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [4]:
queries = processQueries()

In [5]:
len(queries)

1418

In [6]:
mongo = m.MongoDB("Queries")

In [9]:
just_one = take(1, queries.items())
mongo.update_one_query(({'queryNum': just_one[0][0][0]}, {'test': 'test2'}))
print(mongo.read_one_query({'queryNum': just_one[0][0][0]}))

({'queryNum': 1}, {'test': 'test2'})
Query 5eaf1fef334f483dd6bb9e34 has been updated
{'_id': ObjectId('5eaf1fef334f483dd6bb9e34'), 'queryNum': 1, 'questionId': 313665, 'variantId': 4388082, 'rawQueryContent': "select e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle\nfrom Employee e natural join Department d\nwhere (d.deptLocation = 'Dubai' or d.deptName = 'Sales') and\n        (e.annualSalary >= 7000 and e.annualSalary <= 100000) and\n        (e.empName like '_a%' or e.empName like '_i%')\norder by e.empName asc, d.deptName desc", 'test': 'test2'}


In [52]:
# Try with one query
curr = queries[1]
parsed = sqlparse.parse(curr)
print(parsed[0])
print(parsed[0].tokens)

for tok in parsed[0].tokens:
    # print(tok.ttype, type(tok))
    if (type(tok) == sqlparse.sql.Identifier):
        print(tok.value)

select e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle
from Employee e natural join Department d
where (d.deptLocation = 'Dubai' or d.deptName = 'Sales') and
        (e.annualSalary >= 7000 and e.annualSalary <= 100000) and
        (e.empName like '_a%' or e.empName like '_i%')
order by e.empName asc, d.deptName desc
[<DML 'select' at 0x10E1B7D00>, <Whitespace ' ' at 0x10E1B7980>, <IdentifierList 'e.empN...' at 0x10E1C39D0>, <Newline ' ' at 0x10E1B7360>, <Keyword 'from' at 0x10E1B7EC0>, <Whitespace ' ' at 0x10E1B76E0>, <Identifier 'Employ...' at 0x10E1C37D0>, <Whitespace ' ' at 0x10E1B5AD0>, <Keyword 'natura...' at 0x10E1B7F30>, <Whitespace ' ' at 0x10E1B5A60>, <Identifier 'Depart...' at 0x10E1C3B50>, <Newline ' ' at 0x10E1B5360>, <Where 'where ...' at 0x10E1C33D0>, <Keyword 'order ...' at 0x10E1A2210>, <Whitespace ' ' at 0x10E1A21A0>, <IdentifierList 'e.empN...' at 0x10E1B1150>]
Employee e
Department d


In [3]:
mongo.read_queries({})

In [4]:
mongo.create_query({"queryid": 1, "value": "test"})

Inserted "5eaf10f0334f483dd6bb9e33" for {'queryid': 1, 'value': 'test', '_id': ObjectId('5eaf10f0334f483dd6bb9e33')}


In [49]:
for i in mongo.read_queries({}):
    print(i)

{'_id': ObjectId('5eaf1fef334f483dd6bb9e34'), 'queryNum': 1, 'questionId': 313665, 'variantId': 4388082, 'rawQueryContent': "select e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle\nfrom Employee e natural join Department d\nwhere (d.deptLocation = 'Dubai' or d.deptName = 'Sales') and\n        (e.annualSalary >= 7000 and e.annualSalary <= 100000) and\n        (e.empName like '_a%' or e.empName like '_i%')\norder by e.empName asc, d.deptName desc", 'test': 'test2', 'parsedRemovedQuery': "select empName, annualSalary, deptName, deptLocation, jobTitle\nfrom Employee natural join Department\nwhere (deptLocation = 'Dubai' or deptName = 'Sales') and\n        (annualSalary >= 7000 and annualSalary <= 100000) and\n        (empName like '_a%' or empName like '_i%')\norder by empName asc, deptName desc"}
{'_id': ObjectId('5eaf1fef334f483dd6bb9e35'), 'queryNum': 2, 'questionId': 34820, 'variantId': 4388082, 'rawQueryContent': "SELECT e.empName, e.annualSalary, d.deptName, d.deptL

{'_id': ObjectId('5eaf1ff2334f483dd6bb9e99'), 'queryNum': 102, 'questionId': 33032, 'variantId': 3902466, 'rawQueryContent': "Select empID, empName, jobTitle,managerID, hireDate, salary, comission, e.deptID\nFrom Employee e, Department d, Salary_Grade s\nWhere s.grade IN (12,13)\nAND e.salary Between s.minSalary And s.maxSalary\nAND e.deptID=d.deptID\nAND d.deptLocation = 'London'", 'parsedRemovedQuery': "Select empID, empName, jobTitle,managerID, hireDate, salary, comission, deptID\nFrom Employee, Department, Salary_Grade\nWhere grade IN (12,13)\nAND salary Between minSalary And maxSalary\nAND deptID=deptID\nAND deptLocation = 'London'"}
{'_id': ObjectId('5eaf1ff2334f483dd6bb9e9a'), 'queryNum': 103, 'questionId': 34602, 'variantId': 3902466, 'rawQueryContent': "SELECT *\nFROM Employee\nWHERE deptID = (SELECT deptID FROM Department WHERE deptLocation = 'London')\nAND salary >= (SELECT minSalary FROM Salary_Grade WHERE grade = 12) AND salary <= (SELECT maxSalary FROM Salary_Grade WHERE 

In [10]:
for i in mongo.read_queries({}):
    print(i)

In [47]:
def parse_remove(queries):
    for query in queries:
        content = query[1]
        parsed = sqlparse.parse(query[1])
        
        prev_keyword = ""
        for tok in parsed[0].tokens:
            if(str(tok.ttype) == "Token.Keyword"):
                prev_keyword = tok.value.lower()
            if(type(tok) == sqlparse.sql.Identifier) and (prev_keyword == "from" or "join" in prev_keyword):
                # Process tok.value
                # The order of Department vs. Employee shouldn't matter
                alias = tok.value.split(" ")
                print(alias)
                if(len(alias) > 1 and (alias[-1].lower != "asc" or alias[-1].lower != "desc")):
                    content = content.replace(" " + alias[-1] + " ", " ")
                    content = content.replace(" as " + alias[-1] + " ", " ")
                    content = content.replace(" as " + alias[-1] + "\n", "\n")
                    content = content.replace(" " + alias[-1] + "\n", "\n")
                    content = content.replace(alias[-1] + ".", "")

            if(type(tok) == sqlparse.sql.IdentifierList):
                # use regex to make sure is alias one
                if re.match(r'[a-zA-Z ]+ [a-zA-Z ]+,*', tok.value) or re.match(r'[a-zA-Z .]+ as [a-zA-Z ]+,*', tok.value):
                    
                    sets = tok.value.split(",")
                    for s in sets:
                        alias = s.strip().split(" ")
                        print(alias)
                        if(len(alias) > 1 and (alias[-1].lower() != "asc" and alias[-1].lower() != "desc" and "." not in alias[-1])):
                            if len(alias) == 2:
                                content = content.replace(" " + alias[-1] + ",", ",")
                                content = content.replace(" " + alias[-1] + "\n", "\n")
                                content = content.replace(alias[-1] + ".", "")
                            else:
                                as_flag = False
                                for i in alias:
                                    if as_flag:
                                        # this is an alias
                                        content = content.replace(" as " + i + ",", ",")
                                        content = content.replace(" as " + i + " ", " ")
                                        content = content.replace(" as " + i + "\n", "\n")
                                        content = content.replace(i + ".", "")
                                        as_flag = False
                                    if i == "as":
                                        as_flag = True
        mongo.update_one_query(({'queryNum': query[0][0]}, {'parsedRemovedQuery': content}))

        print("==============")

In [48]:
parse_remove(queries.items())

['Employee', 'e']
['Department', 'd']
({'queryNum': 1}, {'parsedRemovedQuery': "select empName, annualSalary, deptName, deptLocation, jobTitle\nfrom Employee natural join Department\nwhere (deptLocation = 'Dubai' or deptName = 'Sales') and\n        (annualSalary >= 7000 and annualSalary <= 100000) and\n        (empName like '_a%' or empName like '_i%')\norder by empName asc, deptName desc"})
Query 5eaf1fef334f483dd6bb9e34 has been updated
['Employee', 'e']
['Department', 'd']
({'queryNum': 2}, {'parsedRemovedQuery': "SELECT empName, annualSalary, deptName, deptLocation, jobTitle\nFROM Employee join Department on deptID = deptID \nWHERE ((deptLocation = 'Dubai') OR (deptName = 'Sales')) AND (annualSalary >= 84000 AND annualSalary <= 120000) AND ((empName LIKE '_A%' )OR (empName LIKE '_I%'))\nORDER BY empName,  deptName DESC\n\n"})
Query 5eaf1fef334f483dd6bb9e35 has been updated
['Employee', 'e']
['Department', 'd']
({'queryNum': 3}, {'parsedRemovedQuery': "SELECT empName, annualSalary, 

Query 5eaf1fef334f483dd6bb9e48 has been updated
['Employee']
['Department']
({'queryNum': 22}, {'parsedRemovedQuery': "select Employee.empName, Employee.annualSalary, Department.deptName,Department.deptLocation, Employee.jobTitle\nfrom Employee natural join Department\nwhere (Department.deptLocation = 'Dubai' or deptName = 'Sales') and Employee.annualSalary/12>=7000 and Employee.annualSalary/12<= 10000 and (Employee.empName like '_A%' or Employee.empName like '_I%')\norder by Employee.empName, Department.deptName desc"})
Query 5eaf1fef334f483dd6bb9e49 has been updated
['Employee', 'E']
['Department', 'D']
({'queryNum': 23}, {'parsedRemovedQuery': "select empName, annualSalary, deptName, deptLocation, jobTitle\nfrom Employee join Department on deptID = deptID\nwhere (deptLocation = 'Dubai' or deptName = 'Sales') and ((annualSalary/12)>= 7000 and (annualSalary/12) <= 10000) and (empName LIKE '_A%' or empName LIKE '_I%')\norder by empName, deptName desc"})
Query 5eaf1fef334f483dd6bb9e4a h

Query 5eaf1ff0334f483dd6bb9e5c has been updated
['Employee', 'E']
['Department', 'D']
['Salary_Grade', 'S']
({'queryNum': 42}, {'parsedRemovedQuery': "Select empName,annualSalary,deptName,deptLocation,jobTitle\nfrom Employee,Department,Salary_Grade\nwhere deptID=deptID and annualSalary >= minAnnualSalary and annualSalary <= maxAnnualSalary and\n(deptLocation LIKE 'Dubai' OR deptName LIKE 'Sales') and \n(annualSalary/12 >= 7000 and annualSalary/12 <= 10000) and\n(empName LIKE '_A%' OR empName LIKE '_I%')\norder by empName,deptName DESC"})
Query 5eaf1ff0334f483dd6bb9e5d has been updated
['Employee', 'e']
['Department', 'd']
({'queryNum': 43}, {'parsedRemovedQuery': 'SELECT empName, annualSalary, deptName, deptLocation, jobTitle \nFROM Employee, Department\nWHERE deptId = deptId \n    AND (deptLocation = "Dubai" OR deptName = "Sales")\n    AND ((empName LIKE "_a%") OR (empName LIKE "_i%"));\n    '})
Query 5eaf1ff0334f483dd6bb9e5e has been updated
['Employee', 'e']
['Department', 'd']
({'q

Query 5eaf1ff1334f483dd6bb9e73 has been updated
['Salary_Grade', 'sg']
({'queryNum': 65}, {'parsedRemovedQuery': 'select ee.empName, ee.jobTitle, ee.salary/12, ee.deptID, deptName, grade\nFROM (Employee ee JOIN Department de ON ee.deptID = de.deptID) JOIN Salary_Grade ON ee.salary BETWEEN minSalary AND maxSalary\nWHERE ee.salary/12 > 7000 OR NOT (ee.jobTitle = "IT Support")'})
Query 5eaf1ff1334f483dd6bb9e74 has been updated
['Employee', 'e']
['Department', 'd']
['Salary_Grade', 's']
({'queryNum': 66}, {'parsedRemovedQuery': "SELECT empName, jobTitle, salary / 12, deptID, deptName, grade\nFROM Employee, Department, Salary_Grade\nWHERE deptID = deptID\nAND salary >= minSalary AND salary <= maxSalary\nAND (salary/12 > 7000 OR empID NOT IN\n(SELECT empID\nFROM Employee\nWHERE jobTitle <> 'IT Support'))"})
Query 5eaf1ff1334f483dd6bb9e75 has been updated
['Employee', 'e']
['Department', 'd']
['Salary_Grade', 's']
({'queryNum': 67}, {'parsedRemovedQuery': "SELECT empName, jobTitle, salary/12,

Query 5eaf1ff1334f483dd6bb9e89 has been updated
['Employee', 'e']
['Department', 'd']
['Salary_Grade', 's']
({'queryNum': 87}, {'parsedRemovedQuery': 'SELECT empName, jobTitle, salary/12 AS MonthlySal, deptId, deptName, grade\nFROM Employee, Department, Salary_Grade\nWHERE (deptId = deptId) AND (salary >= minSalary AND salary <= maxSalary) AND (salary/12 > 7000 OR jobTitle != "IT Support")'})
Query 5eaf1ff1334f483dd6bb9e8a has been updated
['Employee', 'e']
['Department', 'd']
['Salary_Grade', 'sg']
({'queryNum': 88}, {'parsedRemovedQuery': 'SELECT empName, jobTitle, salary/12 AS MonthlySal, deptID, deptName, grade\nFROM Employee NATURAL JOIN Department, Salary_Grade\nWHERE (jobTitle <> "IT Support" OR (salary/12) > 7000) AND salary BETWEEN minSalary AND maxSalary'})
Query 5eaf1ff1334f483dd6bb9e8b has been updated
['Employee', 'E']
['Salary_Grade', 'S']
['Department', 'D']
({'queryNum': 89}, {'parsedRemovedQuery': "SELECT empName, jobTitle, (salary/12) as monthlySalary, deptID, deptNam

...

Query 5eaf2013334f483dd6bba35e has been updated
['Products', 'p1']
['Products', 'p2']
({'queryNum': 1324}, {'parsedRemovedQuery': 'SELECT ProductName, SUM(YearReleased > YearReleased)\nFROM Products LEFT OUTER JOIN Products ON BrandName = BrandName\nGROUP BY ProductName'})
Query 5eaf2013334f483dd6bba35f has been updated
['Products']
({'queryNum': 1325}, {'parsedRemovedQuery': 'SELECT new.ProductName, COUNT(old.ProductId)-1\nFROM Products new, Products old\nWHERE (new.YearReleased > old.YearReleased and new.BrandName = old.BrandName) or (new.YearReleased = old.YearReleased and new.ProductName = old.productName)\nGROUP BY new.ProductName'})
Query 5eaf2013334f483dd6bba360 has been updated
['Products', 'as', 'P1']
({'queryNum': 1326}, {'parsedRemovedQuery': 'SELECT ProductName, \n(SELECT COUNT(*) FROM Products as P2 \nWHERE BrandName = P2.BrandName \nAND P2.YearReleased < YearReleased)\nFROM Products\nORDER BY ProductName'})
Query 5eaf2013334f483dd6bba361 has been updated
['Products']
({'q

Query 5eaf2014334f483dd6bba375 has been updated
['Products', 'p']
({'queryNum': 1347}, {'parsedRemovedQuery': 'SELECT ProductName, (SELECT COUNT(*)\n                    FROM Products a\n                    WHERE YearReleased > a.YearReleased AND BrandName = a.BrandName)\n                    \nFROM Products\nORDER BY ProductName\n'})
Query 5eaf2014334f483dd6bba376 has been updated
['Products', 'AS', 'p']
['Products', 'AS', 'old']
({'queryNum': 1348}, {'parsedRemovedQuery': 'SELECT\nProductName,\nSUM(CASE WHEN YearReleased > YearReleased then 1 else 0 end) as Products_released_before\nFROM Products AS\nLEFT JOIN Products AS ON YearReleased > YearReleased AND BrandName = BrandName\ngroup by ProductName;'})
Query 5eaf2014334f483dd6bba377 has been updated
['Products', 'p1']
['Products', 'p2\n--', 'on', '', '\n--', 'p1.ProductId', '<>', 'p2.ProductId']
({'queryNum': 1349}, {'parsedRemovedQuery': 'Select  ProductName,  Count(p2.ProductId) - 1\nFrom Products,  Products p2\n-- on  \n-- ProductI

Query 5eaf2014334f483dd6bba38d has been updated
['Products', 'p']
({'queryNum': 1371}, {'parsedRemovedQuery': 'select ProductName, (select count(*) from Products where YearReleased < YearReleased and BrandName = BrandName) as cnt from Products order by ProductName'})
Query 5eaf2014334f483dd6bba38e has been updated
['Products']
({'queryNum': 1372}, {'parsedRemovedQuery': 'SELECT ProductName, (\n    SELECT COUNT(ProductName)\n    FROM Products AS ProductsBefore\n    WHERE Products.YearReleased > ProductsBefore.YearReleased \n        AND Products.BrandName = ProductsBefore.BrandName\n)\nFROM Products\nORDER BY ProductName ASC; '})
Query 5eaf2014334f483dd6bba38f has been updated
['Products', 'd']
({'queryNum': 1373}, {'parsedRemovedQuery': 'SELECT ProductName, (SELECT COUNT(*)-1 FROM Products p2 WHERE ((p2.YearReleased <YearReleased AND BrandName = p2.BrandName) OR (p2.ProductId = ProductId AND p2.YearReleased=YearReleased)) GROUP by p2.BrandName)\nFROM Products\nGROUP By BrandName, Produc

Query 5eaf2015334f483dd6bba3a3 has been updated
['Products', 'p1']
['Products', 'p2']
({'queryNum': 1393}, {'parsedRemovedQuery': 'select ProductName, sum(if(YearReleased < YearReleased, 1, 0))\nfrom Products inner join Products\non BrandName = BrandName\ngroup by ProductName\norder by ProductName\n'})
Query 5eaf2015334f483dd6bba3a4 has been updated
['Products', 'AS', 'A']
['Products', 'AS', 'B']
({'queryNum': 1394}, {'parsedRemovedQuery': 'SELECT A.ProductName, COUNT(CASE WHEN (A.BrandName = B.BrandName AND A.YearReleased > B.YearReleased) THEN 1 ELSE NULL END) AS PrevProducts\nFROM Products AS A, Products AS B\nGROUP BY A.ProductName;'})
Query 5eaf2015334f483dd6bba3a5 has been updated
['Products', 'p1']
({'queryNum': 1395}, {'parsedRemovedQuery': 'SELECT ProductName, (SELECT COUNT(ProductId) FROM Products p2 WHERE p2.BrandName=BrandName AND p2.YearReleased<YearReleased)\nFROM Products\nORDER BY ProductName ASC'})
Query 5eaf2015334f483dd6bba3a6 has been updated
['(\n', '', '', '', 'SE

Query 5eaf2015334f483dd6bba3b4 has been updated
['Products', 'p1']
['Products', 'p2']
({'queryNum': 1410}, {'parsedRemovedQuery': 'Select ProductName, count(ProductName)\nfrom Products left outer join Products on BrandName = BrandName\nand YearReleased < YearReleased\ngroup by ProductName'})
Query 5eaf2016334f483dd6bba3b5 has been updated
['Products', 'm']
['Products', 'n']
({'queryNum': 1411}, {'parsedRemovedQuery': 'select ProductName, Count(*)-1\nfrom Products join Products \nwhere BrandName = BrandName and (YearReleased > YearReleased or ProductName = ProductName)\ngroup by ProductName\n'})
Query 5eaf2016334f483dd6bba3b6 has been updated
['Products', 'p1']
['Products', 'p2']
({'queryNum': 1412}, {'parsedRemovedQuery': 'SELECT ProductName, SUM(CASE WHEN YearReleased > YearReleased && BrandName = BrandName THEN 1 ELSE 0 END)\nFROM Products, Products\nGROUP BY ProductName'})
Query 5eaf2016334f483dd6bba3b7 has been updated
['Products', 'p']
({'queryNum': 1413}, {'parsedRemovedQuery': '

In [10]:
# For first ten queries, let's substitute:
def parse(queries):
    aliases = ['a', 'b', 'c', 'd', 'e', 'f']
    parsed_queries = []
    # f = open("parsed_queries.csv", "w")
    mongo = m.MongoDB("Queries")
    
    for query in queries:
        #print("New Query\n")
        content = query[1]
        substitute_alias = ""
        remove_alias = ""
        substitute_all = ""
        #print(content)
        curr_alias = 0
        
        parsed = sqlparse.parse(query[1])
        
        #if len(parsed) == 1:
        prev_keyword = ""
        for tok in parsed[0].tokens:
            # print(type(tok), tok.ttype, tok.value)
            if(str(tok.ttype) == "Token.Keyword"):
                prev_keyword = tok.value.lower()
            if(type(tok) == sqlparse.sql.Identifier) and (prev_keyword == "from" or "join" in prev_keyword):
                # Process tok.value
                # The order of Department vs. Employee shouldn't matter
                alias = tok.value.split(" ")
                #print(alias)
                if(len(alias) > 1 and (alias[-1].lower != "asc" or alias[-1].lower != "desc")):
                    content = content.replace(" " + alias[-1] + " ", " " + aliases[curr_alias] + " ")
                    content = content.replace(alias[-1] + ".", aliases[curr_alias] + ".")
                    curr_alias += 1

            if(type(tok) == sqlparse.sql.IdentifierList):
                # use regex to make sure is alias one
                if re.match(r'[a-zA-Z ]+ [a-zA-Z ]+,*', tok.value):
                    sets = tok.value.split(",")
                    for s in sets:
                        alias = s.strip().split(" ")
                        #print(alias)
                        if(len(alias) > 1 and (alias[-1].lower() != "asc" and alias[-1].lower() != "desc" and "." not in alias[-1])):
                            if len(alias) == 2:
                                content = content.replace(" " + alias[-1] + ",", " " + aliases[curr_alias] + ",")
                                content = content.replace(" " + alias[-1] + "\n", " " + aliases[curr_alias] + "\n")
                                content = content.replace(alias[-1] + ".", aliases[curr_alias] + ".")
                            else:
                                as_flag = False
                                for i in alias:
                                    if as_flag:
                                        # this is an alias
                                        content = content.replace(" " + i + ",", " " + aliases[curr_alias] + ",")
                                        content = content.replace(" " + i + "\n", " " + aliases[curr_alias] + "\n")
                                        content = content.replace(i + ".", aliases[curr_alias] + ".")
                                        as_flag = False
                                    if i == "as":
                                        as_flag = True
                            curr_alias += 1
                            
        print(query[0][0], query[0][1], query[0][2], content)
        #f.write(str(query[0][0]) + ", " + str(query[0][1]) + "," + str(query[0][2]) + "," + content)
        #f.write("\n")
                    
        #else:
        #    print("more than one")
        #    for p in parsed:
        #        print(p, "boop")
        print("=============")
    return parsed_queries

In [11]:
n = 10
subset = take(n, queries.items())
print(subset)

[((1, 313665, 4388082), "select e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle\nfrom Employee e natural join Department d\nwhere (d.deptLocation = 'Dubai' or d.deptName = 'Sales') and\n        (e.annualSalary >= 7000 and e.annualSalary <= 100000) and\n        (e.empName like '_a%' or e.empName like '_i%')\norder by e.empName asc, d.deptName desc"), ((2, 34820, 4388082), "SELECT e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle\nFROM Employee e join Department d on e.deptID = d.deptID \nWHERE ((d.deptLocation = 'Dubai') OR (d.deptName = 'Sales')) AND (e.annualSalary >= 84000 AND e.annualSalary <= 120000) AND ((e.empName LIKE '_A%' )OR (e.empName LIKE '_I%'))\nORDER BY e.empName,  d.deptName DESC\n\n"), ((3, 325288, 4388082), "SELECT e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle\nFROM Employee e NATURAL JOIN Department d\nWHERE (d.deptLocation = 'Dubai' OR d.deptName = 'Sales') AND e.annualSalary >= 84000 AND e.annualSalary <= 120000

In [12]:
parse(subset)

1 313665 4388082 select a.empName, a.annualSalary, b.deptName, b.deptLocation, a.jobTitle
from Employee a natural join Department d
where (b.deptLocation = 'Dubai' or b.deptName = 'Sales') and
        (a.annualSalary >= 7000 and a.annualSalary <= 100000) and
        (a.empName like '_a%' or a.empName like '_i%')
order by a.empName asc, b.deptName desc
2 34820 4388082 SELECT a.empName, a.annualSalary, b.deptName, b.deptLocation, a.jobTitle
FROM Employee a join Department b on a.deptID = b.deptID 
WHERE ((b.deptLocation = 'Dubai') OR (b.deptName = 'Sales')) AND (a.annualSalary >= 84000 AND a.annualSalary <= 120000) AND ((a.empName LIKE '_A%' )OR (a.empName LIKE '_I%'))
ORDER BY a.empName,  b.deptName DESC


3 325288 4388082 SELECT a.empName, a.annualSalary, b.deptName, b.deptLocation, a.jobTitle
FROM Employee a NATURAL JOIN Department d
WHERE (b.deptLocation = 'Dubai' OR b.deptName = 'Sales') AND a.annualSalary >= 84000 AND a.annualSalary <= 120000 AND (a.empName LIKE '_A%' OR a.empName 

[]

In [212]:
par_queries = parse(queries.items())

In [192]:
parse([list(queries.items())[-2]])

more than one
SELECT p1.ProductName, COUNT(p2.YearReleased) - 1
FROM Products as p1, Products as p2
WHERE (p1.BrandName = p2.BrandName AND p2.YearReleased < p1.YearReleased) OR
      (p1.ProductName = p2.ProductName)
GROUP BY p1.ProductName; boop

 boop


[]

In [179]:
par_queries[0]

(1,
 "select a.empName, a.annualSalary, b.deptName, b.deptLocation, a.jobTitle\nfrom Employee a natural join Department d\nwhere (b.deptLocation = 'Dubai' or b.deptName = 'Sales') and\n        (a.annualSalary >= 7000 and a.annualSalary <= 100000) and\n        (a.empName like '_a%' or a.empName like '_i%')\norder by a.empName asc, b.deptName desc")

In [198]:
par_queries[-2]

(1417,
 'SELECT a.ProductName, COUNT(b.YearReleased) - 1\nFROM Products as a, Products as b\nWHERE (a.BrandName = b.BrandName AND b.YearReleased < a.YearReleased) OR\n      (a.ProductName = b.ProductName)\nGROUP BY a.ProductName;\n')

In [196]:
len(par_queries)

1418